In [1]:
!pip install -U pyupbit
!pip install PublicDataReader
!pip install quart
! pip install Flask

     ---------------------------------------- 3.2/3.2 MB 10.8 MB/s eta 0:00:00
     -------------------------------------- 100.2/100.2 kB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 57.9/57.9 kB ? eta 0:00:00
     ------------------------------------- 233.6/233.6 kB 14.0 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 2.0.3
    Uninstalling Werkzeug-2.0.3:
      Successfully uninstalled Werkzeug-2.0.3


In [2]:
Python 3.8.0 64-bit('env':venv)

SyntaxError: invalid syntax (1971025180.py, line 1)

In [ ]:
test_duplicate_password()

In [ ]:
# Flask의 비동기 모드를 활성화 필요 조건
!pip install flask>=2.0


#애플리케이션을 생성할 때, async_mode를 활성화합니다.

In [ ]:
from flask import Flask, render_template, request, redirect
from PublicDataReader import TransactionPrice
import PublicDataReader as pdr

service_key = "gheSnUnob3mSKc5nus5HfTB5tebwjYj14Y28AM9Fw%2F%2BcET6pE%2Bmsfo1BY7ot0KX5fsuR7RsLDWZYxzvZ7UVOcg%3D%3D"

app = Flask(__name__)

@app.route("/")
def index():
    return '''
    <!DOCTYPE html>
    <html>
        <head>
            <title>Input Text</title>
        </head>
        <body>
            <h1>부동산 주소검색</h1>
            <form action="/Create/" method="POST">
                <p><input type="text" name="Search" placeholder="서울특별시 서초구 서초4동 1685"></p>
                로딩하는 동안 시간이 걸릴수 있으니 다음 작업을 진행해주세요
                <p><input type="submit" value="Create"></p>
            </form>
        </body>
    </html>
    '''

@app.route("/Create/", methods=['GET', 'POST'])
def create():
    if request.method == 'POST':
        search = request.form['Search']
        api = TransactionPrice(service_key)

        address = search
        split_address = address.split()

        sigungu_name = split_address[1]
        code = pdr.code_bdong()
        filtered_code = code.loc[(code['시군구명'].str.contains(sigungu_name)) & (code['읍면동명'] == '')]

        # 법정동 코드의 앞 다섯 글자만 가져옵니다.
        law_dong_code = str(filtered_code['법정동코드'].values[0])[:5]

        results = []

        # 부동산종류
        property_types = ["아파트", "오피스텔", "단독다가구", "연립다세대", "토지", "분양입주권", "공장창고"]
        selected_columns = ['도로명', '법정동', '지번', '아파트', '건축년도', '층', '전용면적']

        for property_type in property_types:
            try:
                df = api.get_data(
                    property_type=property_type,
                    trade_type="매매",
                    sigungu_code=law_dong_code,
                    start_year_month="202201",
                    end_year_month="202212",
                )

                if not df.empty:
                    # 지번이 같은 데이터만 필터링합니다.
                    filtered_df = df[df['지번'] == split_address[3]][selected_columns]
                    
                    if not filtered_df.empty:
                        results.append(f"{property_type} 데이터:")
                        for index, row in filtered_df.iterrows():
                            row_data = " | ".join(row.astype(str).tolist())
                            results.append(f'<option>{row_data}</option>')
                        results.append("<br>")
            except:
                pass

        results.append("<br>")
    else:
        results = ["검색 결과가 없습니다."]

    result_string = "".join(results)

    return '''
    <!DOCTYPE html>
    <html>
        <head>
            <title>Real Estate Search Results</title>
        </head>
        <body>
        <form action="/selected_data/" method="POST">
            <h1>부동산 주소검색 결과</h1>
            <div>
                <select>
                <option disabled>   도로명   | 법정동 |  지번  |   아파트   | 건축년도 | 층 | 전용면적</option>
                {}
                </select>
            </div>
            <p><input type="submit" value="Submit"></p>
            </form>
            <a href="/">다시 검색하려면 여기를 클릭하세요.</a>
        </body>
    </html>
    '''.format("".join(results))

@app.route("/selected_data/", methods=["POST"])
def selected_data():
    global selected_result
    if request.method == "POST":
        selected_result = request.form["selected_result"]
        return redirect("/get_selected_data/")
    return "Method not allowed", 405

@app.route("/get_selected_data/")
def get_selected_data():
    global selected_result
    return selected_result


if __name__ == "__main__":
    app.run(debug=False)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Apr/2023 16:27:42] "GET / HTTP/1.1" 200 -


출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2023. 2. 24. 시행)
URL: https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=98870



127.0.0.1 - - [17/Apr/2023 16:27:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 16:27:47] "POST /Create/ HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 16:27:52] "POST /selected_data/ HTTP/1.1" 400 -
127.0.0.1 - - [17/Apr/2023 16:28:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 16:28:09] "GET / HTTP/1.1" 200 -


출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2023. 2. 24. 시행)
URL: https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=98870



127.0.0.1 - - [17/Apr/2023 16:28:15] "POST /Create/ HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 16:28:31] "POST /selected_data/ HTTP/1.1" 400 -
127.0.0.1 - - [17/Apr/2023 16:29:33] "GET / HTTP/1.1" 200 -


출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2023. 2. 24. 시행)
URL: https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=98870



127.0.0.1 - - [17/Apr/2023 16:29:38] "POST /Create/ HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 16:29:41] "POST /selected_data/ HTTP/1.1" 400 -
